In [ ]:
import json
import os
from transformers import RobertaTokenizer, EncoderDecoderModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
def load_bugs_data(path):
    with open(path, 'r') as f:
        return json.load(f)

def init_model():
    tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
    model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
    return tokenizer, model


def print_bug_info(bug):
    print(f"\n🐞 Bug in project: {bug['projectName']}")
    print(f"📄 File: {bug['bugFilePath']}")
    print(f"🔢 Bug Line: {bug['bugLineNum']}")
    print(f"❌ Before: {bug['sourceBeforeFix']}")
    print(f"✅ After:  {bug['sourceAfterFix']}")


def generate_fix_with_model(buggy_line, tokenizer, model):
    input_text = f"fix: {buggy_line}"
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)
    outputs = model.generate(inputs.input_ids, max_length=256, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def process_bugs(json_path):
    bugs = load_bugs_data(json_path)
    tokenizer, model = init_model()

    for bug in bugs[:5]:
        print_bug_info(bug)
        buggy_code = bug['sourceBeforeFix']
        fixed_code = bug['sourceAfterFix']
        model_fix = generate_fix_with_model(buggy_code, tokenizer, model)

        print("\n Model-generated fix suggestion:")
        print(model_fix)

        print("\n Ground-truth fix from dataset:")
        print(fixed_code)

if __name__ == "__main__":
    process_bugs("/content/drive/MyDrive/bugs")



🐞 Bug in project: Activiti.Activiti
📄 File: modules/activiti-secure-service-tasks/src/test/java/org/activiti/test/serviceTask/secure/SecureShellTaskTest.java
🔢 Bug Line: 41
❌ Before: "shellCommandStart"
✅ After:  "shellCommandEcho"

 Model-generated fix suggestion:
fix: "shellCommandEnd" fix : "shellCommandEnd"

 Ground-truth fix from dataset:
"shellCommandEcho"

🐞 Bug in project: Activiti.Activiti
📄 File: modules/activiti-form-engine/src/main/java/org/activiti/form/engine/impl/cmd/GetRuntimeFormDefinitionCmd.java
🔢 Bug Line: 184
❌ Before: submittedNode.get("values") != null
✅ After:  submittedNode.get("values") == null

 Model-generated fix suggestion:
: submittedNode.get("values") != null

 Ground-truth fix from dataset:
submittedNode.get("values") == null

🐞 Bug in project: Activiti.Activiti
📄 File: modules/activiti-form-engine/src/main/java/org/activiti/form/engine/impl/cmd/GetRuntimeFormDefinitionCmd.java
🔢 Bug Line: 224
❌ Before: dateValue
✅ After:  dateValue.toString("yyyy-M-d"